# Industrial Internet of Things

![](https://gitlab.com/ch-mc-b/autoshop-ms/infra/iiot/-/raw/main/images/crds.png)

Custom Resources
- - -

Device – IoT Device, z.B. M5StackCore, Atom etc. beinhaltet Sensor und Actor

MQTTDevice empfängt Daten mittels des MQTT-Protokolls. HTTP und Matter Device (* in Planung).

### Custom Resources

Custom Resources erstellen. Diese sind Global im Cluster verfügbar.

In [ ]:
%%bash    
git clone https://gitlab.com/ch-mc-b/autoshop-ms/infra/iiot.git
cd iiot
kubectl apply -f crd

Sensoren, Aktoren und Devices erstellen für die [M5Stack](https://m5stack.com/) IoT Geräte    

In [ ]:
%%bash
cd iiot
kubectl apply -f m5stack/sensors -f m5stack/actors -f m5stack/devices

Ausgabe der erstellten Geräte

In [ ]:
%%bash
kubectl get devices,sensors,actors
echo "--------------------------------------"
kubectl describe sensor enviii   

### Operator deployen

In [ ]:
%%bash
cd iiot
kubectl apply  -f mqtt-operator
kubectl wait --for=condition=Ready pod mqtt-operator --timeout=300s
kubectl get pods

### Eigene MQTTDevices

Um eine MQTT Device zu erstellen, benötigen wir eine YAML Datei.

Dabei sind folgende Werte anpassbar
* **name**: frei wählbar, 
* **deviceRef**: zeigen auf die erstellen Devices (`kubectl get devices`). `m5stack-core` und `m5stack-atom` sind zulässig
* **topic**: wird als erster Eintrag im Topic verwendet, z.B. `mydevice/....`
* **pvcName**: wo werden die Daten gespeichert (`kubectl get persistentvolumeclaim`). `data-claim` das Verzeichnis `/data` auf der VM, `longhorn-rwx` ein longhorn Volume.

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: iiot.mc-b.ch/v1alpha1
kind: MQTTDevice
metadata:
  name: $(cat ~/work/server-ip)
spec:
  deviceRef: m5stack-core  # Name der referenzierten Device-Ressource
  mqttSettings:
    broker: mqtt://cloud.tbz.ch:11883  # URL des MQTT-Brokers
    topic: $(cat ~/work/server-ip)  # MQTT-Topic für das Gerät
    qos: 1  # Quality of Service Level (0, 1 oder 2)
    retain: false  # Nachricht zurückhalten (true oder false)
  storage:
    pvcName: "data-claim"
    mountPath: "/data"
EOF


In [ ]:
%%bash
kubectl logs mqtt-listener-$(cat ~/work/server-ip)

## IIoT Dashboard 

Die Implementierung verfügt über ein Dashboard mit einer Übersicht der IoT Geräte und der Möglichkeit Events an die verschiedenen Topics zu senden.

In [ ]:
%%bash
sed -i -e 's/10.0.40.8/cloud.tbz.ch/g' iiot/mqtt-device-ui/mqtt-device-ui.yaml
cd iiot
kubectl apply  -f mqtt-device-ui    
echo "http://"$(cat ~/work/server-ip)":$(kubectl get service mqtt-device-ui -o=jsonpath='{ .spec.ports[0].nodePort }')/" 

## Aufräumen


In [ ]:
%%bash
cd iiot
kubectl delete -f m5stack/mqtt # --grace-period=0 --force 
kubectl delete mqttdevice $(cat ~/work/server-ip)
kubectl delete -f mqtt-operator # --grace-period=0 --force 
kubectl delete -f mqtt-device-ui # --grace-period=0 --force 
kubectl delete -f m5stack/sensors -f m5stack/actors -f m5stack/devices    
kubectl delete -f crd